# Udemy講座「LangChainによる大規模言語モデル（LLM）アプリケーション開発入門」のコース前半のソースコード

## 2. GPT の API の基礎知識

### 2.4. OpenAI の API キーを発行

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

### 2.5. OpenAI の Completions API にふれる

In [ ]:
import requests
import json

url = "https://api.openai.com/v1/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}
data = {
    "model": "text-davinci-003",
    "prompt": "hello!",
    "temperature": 0,
}

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

### 2.6. OpenAI の Chat API にふれる

In [ ]:
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
}
data = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role": "user", "content": "hello!"}
    ],
    "temperature": 0,
}

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

## 4. LangChain 入門（前半）

### 4.3. Models（LLMs・Chat Models）

In [ ]:
!pip install --quiet langchain==0.0.172 openai==0.27.6

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

result = llm.predict("自己紹介してください。")
print(result)

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

result = chat.predict("自己紹介してください。")
print(result)

### 4.4. Prompts（Prompt Templates）

In [ ]:
from langchain.prompts import PromptTemplate

template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""

prompt = PromptTemplate(
    input_variables=["command"],
    template=template,
)

result = prompt.format(command="ls")
print(result)

### 4.5. Chains

In [ ]:
import langchain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Prompt を用意
template = """
次のコマンドの概要を説明してください。

コマンド: {command}
"""
prompt = PromptTemplate(
    input_variables=["command"],
    template=template,
)

# Chain を作成
chain = LLMChain(llm=chat, prompt=prompt)

# 実行
result = chain.run("ls")
print(result)

In [ ]:
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

# Model を用意
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 1 つ目の Prompt と Chain を用意
cot_template = """
以下の質問に回答してください。

### 質問 ###
{question}
### 質問終了 ###

ステップバイステップで考えましょう。
"""
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template=cot_template,
)
cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

# 2 つ目の Prompt と Chain を用意
summarize_template = """
入力を結論だけ一言に要約してください。

### 入力 ###
{input}
### 入力終了 ###
"""
summarize_prompt = PromptTemplate(
    input_variables=["input"],
    template=summarize_template,
)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

# 2 つの Chain を直列に繋ぐ
cot_summarize_chain = SimpleSequentialChain(
    chains=[cot_chain, summarize_chain])

# 実行
result = cot_summarize_chain(
    "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？")
print(result["output"])

### 4.7. Prompts のより高度な機能（Output Parses）

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from typing import List

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

class Recipe(BaseModel):
    ingredients: List[str] = Field(description="ingredients of the dish")
    steps: List[str] = Field(description="steps to make the dish")

template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = LLMChain(llm=chat, prompt=prompt)

output = chain.run(dish="カレー")
print("=== output ===")
print(output)

recipe = parser.parse(output)
print("=== recipe object ===")
print(recipe)


## 5. LangChain 入門（後半）

### 5.2. Indexes

In [ ]:
!git clone https://github.com/hwchase17/langchain.git

In [ ]:
!cd langchain && git checkout v0.0.172

In [ ]:
!pip install --quiet unstructured==0.6.6 tabulate==0.9.0 pdf2image==1.16.3 pytesseract==0.3.10 chromadb==0.3.23 tiktoken==0.4.0

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator

loader = DirectoryLoader("./langchain/docs/", glob="**/*.md")
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

result = index.query("LangChainの概要を1文で説明してください。", llm=chat)
print(result)

### 5.4. Memory

In [ ]:
def post_chat_completions(content):
  url = "https://api.openai.com/v1/chat/completions"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"]
  }
  data = {
      "model": "gpt-3.5-turbo",
      "messages": [
          {"role": "user", "content": content}
      ],
      "temperature": 0,
  }

  response = requests.post(url=url, headers=headers, json=data)
  print(json.dumps(response.json(), indent=2))

In [ ]:
post_chat_completions("Hi! I'm Oshima!")

In [ ]:
post_chat_completions("Do you know my name?")

In [ ]:
post_chat_completions("""Human: Hi! I'm Oshima!
AI: Hello Oshima! I'm an AI language model. How can I assist you today?
Human: Do you know my name?
AI: """)

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory


langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")

### 5.5. Agents

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"], llm=chat)
agent_chain = initialize_agent(
    tools, chat, agent="zero-shot-react-description")

result = agent_chain.run("What is your current directory?")
print(result)

### 5.8. Chat API に特化した実装

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import openai

langchain.verbose = True
openai.log = "debug"

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    ai_message = conversation.predict(input=user_message)
    print(f"AI: {ai_message}")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi! I'm Oshima!")
]

result = chat(messages)
print(result)

In [ ]:
langchain.verbose = True

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory()

while True:
    user_message = input("You: ")
    memory.chat_memory.add_user_message(user_message)

    ai_message = chat(memory.chat_memory.messages)
    memory.chat_memory.add_ai_message(ai_message.content)
    print(f"AI: {ai_message.content}")
